###Import necessary libraries

In [ ]:
! pip install optuna

In [ ]:
import sklearn
import xgboost

print(f"Scikit-learn version: {sklearn.__version__}")
print(f"XGBoost version: {xgboost.__version__}")


In [ ]:
!pip uninstall scikit-learn -y
!pip install scikit-learn==1.5.2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os,os.path
import re
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import optuna
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils import resample
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from scipy.stats import bootstrap

###Load the datasets

In [ ]:
gene = pd.read_excel('IBD microbes.xlsx')
gene

In [ ]:
gene_ibd = pd.read_excel('IBD microbe validation-iHMP final.xlsx')
gene_ibd

,Group,Limivivens,UBA11774,Ventricola,JAGTTR01,Choladousia,Butyribacter,UMGS1601,RGIG4708,Copromorpha,SIG607,Faecalibaculum,Actinomarina,CAG-83,Bariatricus,UMGS1601.1
0,IBD,0.000291,0.000013,0.000008,0.000005,0.000066,0.000093,0.000000e+00,0.000000,0.000112,0.000000,2.693869e-06,0.000000,0.000190,0.000016,0.000000e+00
1,IBD,0.000089,0.000060,0.000109,0.000094,0.000069,0.002579,2.851185e-06,0.000000,0.000013,0.000000,0.000000e+00,0.000000,0.000286,0.000079,2.851185e-06
2,IBD,0.000021,0.000114,0.000030,0.000019,0.000214,0.000135,1.766807e-06,0.000000,0.000014,0.000002,4.038417e-06,0.000001,0.000351,0.000124,1.766807e-06
3,IBD,0.000012,0.000251,0.000084,0.000013,0.000035,0.000026,1.001865e-06,0.000002,0.000021,0.000000,0.000000e+00,0.000000,0.006983,0.000240,1.001865e-06
4,Healthy,0.000115,0.000353,0.000040,0.000218,0.000199,0.001342,3.430991e-06,0.000002,0.000046,0.000002,1.547310e-06,0.000000,0.005275,0.000825,3.430991e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,IBD,0.000407,0.000028,0.000829,0.000686,0.000242,0.006064,9.453251e-06,0.000004,0.000158,0.000007,7.171432e-07,0.000000,0.008527,0.001889,9.453251e-06
378,IBD,0.000873,0.002969,0.000466,0.000833,0.000659,0.003964,1.246941e-05,0.000011,0.000188,0.000000,1.960599e-06,0.000001,0.008567,0.003799,1.246941e-05
379,IBD,0.000142,0.000034,0.000025,0.000154,0.000168,0.000542,1.905200e-06,0.000000,0.000027,0.000000,6.562356e-06,0.000000,0.000231,0.000495,1.905200e-06
380,IBD,0.000035,0.000209,0.000009,0.000004,0.000020,0.000025,8.838820e-07,0.000004,0.000004,0.000000,6.734339e-07,0.000000,0.000143,0.000020,8.838820e-07


In [ ]:
sample_column = gene['Group']
data_to_scale = gene.drop(columns=['Group'])

scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data_to_scale)

# Convert the scaled data back into a DataFrame and reattach the 'Group' column
gene = pd.DataFrame(data_scaled, columns=data_to_scale.columns)
gene['Group'] = sample_column

# Rearrange the 'Group' column as the first column
gene = gene[['Group'] + [col for col in gene.columns if col != 'Group']]

print("Data after Min-Max scaling:")
print(gene)

In [ ]:
sample_column = gene_ibd['Group']
data_to_scale = gene_ibd.drop(columns=['Group'])

# Apply Min-Max scaling to the data (excluding the 'Group' column)
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data_to_scale)

# Convert the scaled data back into a DataFrame
gene_ibd = pd.DataFrame(data_scaled, columns=data_to_scale.columns)
gene_ibd['Group'] = sample_column

# Rearrange the 'Group' column as the first column
gene_ibd = gene_ibd[['Group'] + [col for col in gene_ibd.columns if col != 'Group']]

print("Data after Min-Max scaling:")
print(gene_ibd)

#XGBOOST

In [ ]:
X = gene.drop(['Group'], axis=1)
y = gene['Group']

# Encode categorical target labels into numerical labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Initialize XGBoost classifier
model = xgb.XGBClassifier(eval_metric='logloss', random_state=42)

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the original test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

# Print evaluation metrics
print(f'Accuracy: {accuracy:.4f}')
print(f'Test Precision: {precision:.4f}')
print(f'Test F1 Score: {f1:.4f}')
print(f'Test Recall: {recall:.4f}')

Accuracy: 0.8727
Test Precision: 0.8703
Test F1 Score: 0.8711
Test Recall: 0.8727


##Random Search

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Define the parameter grid for RandomizedSearchCV
param_dist = {
    'n_estimators': np.arange(50, 200, 10),
    'max_depth': np.arange(3, 10),
    'learning_rate': np.linspace(0.01, 0.3, 10),
    'subsample': np.linspace(0.5, 1.0, 10),
    'colsample_bytree': np.linspace(0.5, 1.0, 10),
    'gamma': np.linspace(0, 0.5, 5),
    'min_child_weight': np.arange(1, 6)
}

# Initialize the XGBoost classifier
xgb = XGBClassifier(random_state=42)

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb, param_distributions=param_dist, n_iter=100,
    scoring='roc_auc', cv=5, verbose=1, random_state=42, n_jobs=-1
)

# Fit the RandomizedSearchCV model on the training data
random_search.fit(X_train, y_train)

# Get the best parameters from the random search
best_params = random_search.best_params_
print("Best Parameters:", best_params)

# Predict on the original test set
y_pred = random_search.predict(X_test)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Print metrics
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best Parameters: {'subsample': 0.7777777777777778, 'n_estimators': 160, 'min_child_weight': 1, 'max_depth': 8, 'learning_rate': 0.07444444444444444, 'gamma': 0.125, 'colsample_bytree': 0.5555555555555556}
Accuracy: 0.87
Precision: 0.87
Recall: 0.87
F1 Score: 0.87


##Bayesian Optimization

In [ ]:
# Define the objective function for Optuna
def objective(trial):
    params = {
        'booster': trial.suggest_categorical('booster', ['gbtree', 'dart']),
        'objective': 'binary:logistic',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 0, 0.5),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.5, 5),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 300)
    }

    # Initialize the XGBoost model with the suggested hyperparameters
    model = xgb.XGBClassifier(**params)

    # Evaluate using cross-validation
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')

    # Return the mean accuracy from cross-validation
    return cv_scores.mean()

# Create a study to maximize accuracy
study = optuna.create_study(direction='maximize')

# Enqueue the parameters obtained from previous RandomizedSearchCV results
study.enqueue_trial({
    'booster': 'gbtree',
    'objective': 'binary:logistic',
    'learning_rate': 0.07444444444444444,
    'gamma':0.125,
    'max_depth': 8,
    'min_child_weight': 1,
    'subsample':0.7777777777777778,
    'colsample_bytree': 0.5555555555555556,
    'n_estimators': 160
})

# Optimize the study using 50 trials
study.optimize(objective, n_trials=50)

# Print the best parameters and cross-validation accuracy
print(f"Best Parameters: {study.best_params}")
print(f"Best Cross-validation Accuracy: {study.best_value:.4f}")

# Train the final model with the best parameters on the training data
best_params = study.best_params
final_model = xgb.XGBClassifier(**best_params)
final_model.fit(X_train, y_train)

# Make predictions on the original test set
y_pred = final_model.predict(X_test)

# Evaluate the final model on the test set
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

# Print test set performance metrics
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test F1 Score: {f1:.4f}")
print(f"Test Recall: {recall:.4f}")

[I 2025-01-08 11:06:22,361] A new study created in memory with name: no-name-62565383-176f-418a-95ff-caf9cae728a6
[I 2025-01-08 11:06:23,187] Trial 0 finished with value: 0.9749466666666666 and parameters: {'booster': 'gbtree', 'learning_rate': 0.07444444444444444, 'gamma': 0.125, 'max_depth': 8, 'min_child_weight': 1.0, 'subsample': 0.7777777777777778, 'colsample_bytree': 0.5555555555555556, 'n_estimators': 160}. Best is trial 0 with value: 0.9749466666666666.
[I 2025-01-08 11:06:31,641] Trial 1 finished with value: 0.9609333333333334 and parameters: {'booster': 'dart', 'learning_rate': 0.24673865411779808, 'gamma': 0.024884626335284432, 'max_depth': 6, 'min_child_weight': 3.177520155001819, 'subsample': 0.8587340395554446, 'colsample_bytree': 0.7669755818029269, 'n_estimators': 190}. Best is trial 0 with value: 0.9749466666666666.
[I 2025-01-08 11:06:35,922] Trial 2 finished with value: 0.9669466666666666 and parameters: {'booster': 'dart', 'learning_rate': 0.043243806396596376, 'gam

Best Parameters: {'booster': 'gbtree', 'learning_rate': 0.17381985302038921, 'gamma': 0.09462296233177214, 'max_depth': 5, 'min_child_weight': 0.573347885974969, 'subsample': 0.731521401285902, 'colsample_bytree': 0.9968978057776777, 'n_estimators': 283}
Best Cross-validation Accuracy: 0.9792
Test Accuracy: 0.8545
Test Precision: 0.8545
Test F1 Score: 0.8545
Test Recall: 0.8545


In [ ]:
selected_features = X_train.columns.tolist()

# Best parameters from Bayesian Optimization XGBoost
best_params_xg = {
    'learning_rate': 0.17381985302038921,
    'max_depth': 5,
    'n_estimators':  283,
    'gamma':0.09462296233177214,
    'min_child_weight': 0.573347885974969,
    'subsample': 0.731521401285902,
    'colsample_bytree':0.9968978057776777,
    'objective': 'binary:logistic',
    'booster': 'gbtree',
}

# Create the XGBoost classifier with the best parameters
final_model_xg = xgb.XGBClassifier(**best_params_xg)

# Perform cross-validation
cv_scores = cross_val_score(final_model_xg, X_train[selected_features], y_train, cv=5, scoring='roc_auc')
print("Cross-validation scores:")
print(cv_scores)
print(f"Mean CV accuracy: {np.mean(cv_scores):.4f}")

# Train the model
final_model_xg.fit(X_train[selected_features], y_train)

# Make predictions on the test set
y_pred_xg = final_model_xg.predict(X_test[selected_features])
y_pred_prob_xg = final_model_xg.predict_proba(X_test[selected_features])[:, 1]

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred_xg)
precision = precision_score(y_test, y_pred_xg, average = 'weighted')
recall = recall_score(y_test, y_pred_xg, average = 'weighted')
f1 = f1_score(y_test, y_pred_xg, average = 'weighted')
roc_auc = roc_auc_score(y_test, y_pred_prob_xg)

# Calculate specificity
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_xg).ravel()
specificity = tn / (tn + fp)

# Print the results
print(f'Test ROC AUC: {roc_auc:.2f}')
print(f'Test Accuracy: {accuracy:.2f}')
print(f'Test Precision: {precision:.2f}')
print(f'Test Recall: {recall:.2f}')
print(f'Test F1-Score: {f1:.2f}')
print(f'Test Specificity: {specificity:.2f}')


Cross-validation scores:
[0.9392     0.99833333 0.99166667 0.97666667 0.99      ]
Mean CV accuracy: 0.9792
Test ROC AUC: 0.86
Test Accuracy: 0.85
Test Precision: 0.85
Test Recall: 0.85
Test F1-Score: 0.85
Test Specificity: 0.71


##XGBoost Validation

In [ ]:
X_val_ibd = gene_ibd.drop('Group', axis=1)
y_val_ibd = gene_ibd['Group']

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_val_ibd)

# Assuming you have the feature names used in training saved
missing_features = [feature for feature in selected_features if feature not in X_val_ibd.columns]

# Add the missing features to the validation set with zero values
missing_df = pd.DataFrame(0.0, index=X_val_ibd.index, columns=missing_features)
X_val_ibd = pd.concat([X_val_ibd, missing_df], axis=1)

# Ensure the columns are in the same order as the training features
X_val_ibd = X_val_ibd[selected_features]

#Make predictions on the validation set
y_pred_prob_xg_ibd= final_model_xg.predict_proba(X_val_ibd)[:, 1]
y_pred_xg_ibd = final_model_xg.predict(X_val_ibd)

# Use the encoded labels for all metrics
y_val_ibd_encoded = label_encoder.transform(y_val_ibd)

# Calculate evaluation metrics
accuracy_val = accuracy_score(y_val_ibd_encoded, y_pred_xg_ibd)
precision_val = precision_score(y_val_ibd_encoded, y_pred_xg_ibd, average="weighted", zero_division=1)
recall_val = recall_score(y_val_ibd_encoded, y_pred_xg_ibd, average="weighted")
f1_val = f1_score(y_val_ibd_encoded, y_pred_xg_ibd, average="weighted")
roc_auc_val = roc_auc_score(y_encoded, y_pred_prob_xg_ibd)

# Calculate specificity
tn_val, fp_val, fn_val, tp_val = confusion_matrix(y_val_ibd_encoded, y_pred_xg_ibd).ravel()
specificity_val = tn_val / (tn_val + fp_val)

# Print evaluation metrics
print(f'Validation ROC AUC: {roc_auc_val:.2f}')
print(f'Validation Accuracy: {accuracy_val:.2f}')
print(f'Validation Precision: {precision_val:.2f}')
print(f'Validation Recall: {recall_val:.2f}')
print(f'Validation F1-Score: {f1_val:.2f}')
print(f'Validation Specificity: {specificity_val:.2f}')


Validation ROC AUC: 0.55
Validation Accuracy: 0.74
Validation Precision: 0.73
Validation Recall: 0.74
Validation F1-Score: 0.74
Validation Specificity: 0.09


##95% CI

In [ ]:
# Function to calculate all metrics
def calculate_metrics(y_true, y_pred, y_pred_prob):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average="weighted", zero_division=1)
    recall = recall_score(y_true, y_pred, average="weighted")
    f1 = f1_score(y_true, y_pred, average="weighted")
    roc_auc = roc_auc_score(y_true, y_pred_prob)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)

    return accuracy, precision, recall, f1, roc_auc, specificity

# Bootstrapping to calculate 95% CI
n_iterations = 1000
metrics = np.zeros((n_iterations, 6))

for i in range(n_iterations):
    X_resampled, y_resampled = resample(X_val_ibd, y_val_ibd_encoded, random_state=i)
    y_pred_prob_resampled = final_model_xg.predict_proba(X_resampled)[:, 1]
    y_pred_resampled = final_model_xg.predict(X_resampled)
    metrics[i] = calculate_metrics(y_resampled, y_pred_resampled, y_pred_prob_resampled)

# Calculate the 2.5th and 97.5th percentiles for each metric
confidence_intervals = np.percentile(metrics, [2.5, 97.5], axis=0)

# Display the metrics with their CIs
metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC AUC', 'Specificity']

for i, name in enumerate(metrics_names):
    print(f'{name}: (95% CI: {confidence_intervals[0, i]:.2f} - {confidence_intervals[1, i]:.2f})')


Accuracy: (95% CI: 0.70 - 0.79)
Precision: (95% CI: 0.65 - 0.80)
Recall: (95% CI: 0.70 - 0.79)
F1-Score: (95% CI: 0.70 - 0.79)
ROC AUC: (95% CI: 0.48 - 0.62)
Specificity: (95% CI: 0.04 - 0.15)


#Random Forest

In [ ]:
X = gene.drop(['Group'], axis=1)
y = gene['Group']

# Encode categorical target labels into numerical labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Create the Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier on the training data
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Calculate accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Calculate F1 score on the test set
test_f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Test F1 Score: {test_f1:.2f}")

# Calculate precision on the test set
test_precision = precision_score(y_test, y_pred, average='weighted')
print(f"Test Precision: {test_precision:.2f}")

# Calculate recall on the test set
test_recall = recall_score(y_test, y_pred, average='weighted')
print(f"Test Recall: {test_recall:.2f}")



Test Accuracy: 0.84
Test F1 Score: 0.84
Test Precision: 0.84
Test Recall: 0.84


##Random Search

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Define the parameter distribution for RandomizedSearchCV
param_dist = {
    'rf__n_estimators': [int(x) for x in np.linspace(start=200, stop=2000, num=10)],
    'rf__max_features': ['sqrt', 'log2'],
    'rf__max_depth': [int(x) for x in np.linspace(10, 300, num=20)] + [None],
    'rf__min_samples_split': [2, 5, 10, 15],
    'rf__min_samples_leaf': [1, 2, 4, 6],
    'rf__bootstrap': [True, False]
}

# Initialize the pipeline: Random Forest
pipeline = Pipeline([
    ('rf', RandomForestClassifier(random_state=42))
])

# Initialize RandomizedSearchCV
rf_random = RandomizedSearchCV(estimator=pipeline, param_distributions=param_dist,
                               n_iter=100, cv=StratifiedKFold(5), verbose=2,
                               random_state=42, n_jobs=-1, scoring='roc_auc')

# Fit RandomizedSearchCV to the original training data
rf_random.fit(X_train, y_train)

# Print the best parameters found by RandomizedSearchCV
print("Best parameters found by RandomizedSearchCV:")
print(rf_random.best_params_)

# Predict on the original test data
y_pred = rf_random.best_estimator_.predict(X_test)
y_prob = rf_random.best_estimator_.predict_proba(X_test)[:, 1]

# Evaluate the model with default threshold
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
roc_auc = roc_auc_score(y_test, y_prob)

# Print evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")


Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best parameters found by RandomizedSearchCV:
{'rf__n_estimators': 200, 'rf__min_samples_split': 10, 'rf__min_samples_leaf': 1, 'rf__max_features': 'sqrt', 'rf__max_depth': None, 'rf__bootstrap': True}
Accuracy: 0.8182
Precision: 0.8281
Recall: 0.8182
F1 Score: 0.8220
ROC AUC: 0.8833


##Bayesian Optimization

In [ ]:
# Define the best parameters found from RandomizedSearchCV RF
best_params_random = {
    'n_estimators': 200,
    'min_samples_split':10,
    'min_samples_leaf':1,
    'max_features': 'sqrt',
    'max_depth': None,
    'bootstrap': True,
    'class_weight': 'balanced'
}

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', max(100, best_params_random['n_estimators'] - 200), best_params_random['n_estimators'] + 200)
    min_samples_split = trial.suggest_int('min_samples_split', max(2, best_params_random['min_samples_split'] - 3), best_params_random['min_samples_split'] + 3)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', max(1, best_params_random['min_samples_leaf'] - 2), best_params_random['min_samples_leaf'] + 2)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2'])
    max_depth = trial.suggest_categorical('max_depth', [None, 10, 25, 162, 300])
    bootstrap = trial.suggest_categorical('bootstrap', [True, False])
    class_weight = trial.suggest_categorical('class_weight', ['balanced', None])
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    min_impurity_decrease = trial.suggest_float('min_impurity_decrease', 0.0, 0.01)

    # Initialize RandomForestClassifier with suggested hyperparameters
    clf = RandomForestClassifier(
        n_estimators=n_estimators,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        max_depth=max_depth,
        bootstrap=bootstrap,
        class_weight=class_weight,
        criterion=criterion,
        min_impurity_decrease=min_impurity_decrease,
        random_state=42
    )

    # Use cross-validation to evaluate the classifier
    cv_scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='roc_auc')
    return np.mean(cv_scores)

# Perform optimization with Optuna
study = optuna.create_study(direction='maximize')

# Enqueue the trial with the best parameters from RandomizedSearchCV
study.enqueue_trial(best_params_random)
study.optimize(objective, n_trials=50)

# Print the best parameters and best score from Optuna
print("Best Parameters from Optuna:", study.best_params)
print("Best F1-Weighted Score from Optuna:", study.best_value)

# Retrieve the best model and evaluate on the test set
best_params_optuna = study.best_params
best_clf = RandomForestClassifier(**best_params_optuna, random_state=50)
best_clf.fit(X_train, y_train)
y_pred = best_clf.predict(X_test)
y_prob = best_clf.predict_proba(X_test)[:, 1]

# Evaluate on test set
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
roc_auc = roc_auc_score(y_test, y_prob)

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1 Score: {f1:.4f}")
print(f"Test ROC AUC: {roc_auc:.4f}")


[I 2025-01-08 12:13:38,560] A new study created in memory with name: no-name-6c0ad1d0-1d46-4567-8835-108bbffcd569
[I 2025-01-08 12:13:40,267] Trial 0 finished with value: 0.9768533333333332 and parameters: {'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': True, 'class_weight': 'balanced', 'criterion': 'entropy', 'min_impurity_decrease': 0.0035874194207065092}. Best is trial 0 with value: 0.9768533333333332.
[I 2025-01-08 12:13:42,142] Trial 1 finished with value: 0.9751733333333334 and parameters: {'n_estimators': 226, 'min_samples_split': 9, 'min_samples_leaf': 3, 'max_features': 'log2', 'max_depth': 10, 'bootstrap': True, 'class_weight': 'balanced', 'criterion': 'entropy', 'min_impurity_decrease': 0.007086237301015272}. Best is trial 0 with value: 0.9768533333333332.
[I 2025-01-08 12:13:45,097] Trial 2 finished with value: 0.9698666666666667 and parameters: {'n_estimators': 368, 'min_samples_split': 11, 'min_

Best Parameters from Optuna: {'n_estimators': 249, 'min_samples_split': 8, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 25, 'bootstrap': False, 'class_weight': 'balanced', 'criterion': 'gini', 'min_impurity_decrease': 0.0014400188813810098}
Best F1-Weighted Score from Optuna: 0.9804666666666666
Test Accuracy: 0.8545
Test Precision: 0.8545
Test Recall: 0.8545
Test F1 Score: 0.8545
Test ROC AUC: 0.8763


In [ ]:
selected_features_rf = X_train.columns.tolist()

# Best parameters from Bayesian Optimization RF
best_params_rf = {
    'n_estimators': 249,
    'min_samples_split': 8,
    'min_samples_leaf': 1,
    'max_features': 'sqrt',
    'bootstrap': False,
    'criterion': 'gini',
    'max_depth': 25,
    'min_impurity_decrease':0.0014400188813810098
}


## Create the Random Forest classifier with the best parameters
final_model_rf = RandomForestClassifier(**best_params_rf, random_state=50)


# Perform cross-validation with 5 folds
cv_scores = cross_val_score(final_model_rf, X_train[selected_features_rf], y_train, cv=5, scoring='roc_auc')
print("Cross-validation scores:")
print(cv_scores)
print(f"Mean CV accuracy: {np.mean(cv_scores):.4f}")

# Train the final model on the entire training data
final_model_rf.fit(X_train[selected_features_rf], y_train)

# Make predictions on the test set
y_pred_rf = final_model_rf.predict(X_test[selected_features_rf])
y_pred_prob_rf = final_model_rf.predict_proba(X_test[selected_features_rf])[:, 1]

# Calculate evaluation metrics on the test set
accuracy = accuracy_score(y_test, y_pred_rf)
precision = precision_score(y_test, y_pred_rf, average = 'weighted')
recall = recall_score(y_test, y_pred_rf, average = 'weighted')
f1 = f1_score(y_test, y_pred_rf, average = 'weighted')
auc_roc = roc_auc_score(y_test, y_pred_prob_rf)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred_rf).ravel()
specificity = tn / (tn + fp)

# Print the results for test data
print(f'Test Accuracy: {accuracy:.2f}')
print(f'Test Precision: {precision:.2f}')
print(f'Test Recall: {recall:.2f}')
print(f'Test F1-Score: {f1:.2f}')
print(f'Test AUC-ROC: {auc_roc:.2f}')
print(f'Test Specificity: {specificity:.2f}')


Cross-validation scores:
[0.9568     0.96       0.99       0.99       0.99833333]
Mean CV accuracy: 0.9790
Test Accuracy: 0.80
Test Precision: 0.80
Test Recall: 0.80
Test F1-Score: 0.80
Test AUC-ROC: 0.86
Test Specificity: 0.57


##Random Forest Validation

In [ ]:
X_val_ibd_rf = gene_ibd.drop('Group', axis=1)
y_val_ibd_rf = gene_ibd['Group']

# Encode categorical target labels into numerical labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_val_ibd_rf)

# Identify the features that are missing in the validation set
missing_features = [feature for feature in selected_features_rf if feature not in X_val_ibd_rf.columns]

# Add the missing features to the validation set with zero values using pd.concat
missing_df_rf = pd.DataFrame(0, index=X_val_ibd_rf.index, columns=missing_features)
X_val_ibd_rf = pd.concat([X_val_ibd_rf, missing_df_rf], axis=1)

# Ensure the columns are in the same order as the training features
X_val_ibd_rf = X_val_ibd_rf[selected_features_rf]

# Make predictions on the validation set without converting to NumPy array
y_pred_prob_rf_ibd = final_model_rf.predict_proba(X_val_ibd_rf)[:, 1]
y_pred_rf_ibd = final_model_rf.predict(X_val_ibd_rf)

# Calculate evaluation metrics
accuracy_val = accuracy_score(y_encoded, y_pred_rf_ibd)
precision_val = precision_score(y_encoded, y_pred_rf_ibd, average='weighted', zero_division=1)
recall_val = recall_score(y_encoded, y_pred_rf_ibd, average='weighted')
f1_val = f1_score(y_encoded, y_pred_rf_ibd, average='weighted')
roc_auc_val = roc_auc_score(y_encoded, y_pred_prob_rf_ibd)

# Calculate specificity
tn_val, fp_val, fn_val, tp_val = confusion_matrix(y_encoded, y_pred_rf_ibd).ravel()
specificity_val = tn_val / (tn_val + fp_val)

# Print evaluation metrics
print(f'Validation ROC AUC: {roc_auc_val:.2f}')
print(f'Validation Accuracy: {accuracy_val:.2f}')
print(f'Validation Precision: {precision_val:.2f}')
print(f'Validation Recall: {recall_val:.2f}')
print(f'Validation F1-Score: {f1_val:.2f}')
print(f'Validation Specificity: {specificity_val:.2f}')



Validation ROC AUC: 0.57
Validation Accuracy: 0.74
Validation Precision: 0.72
Validation Recall: 0.74
Validation F1-Score: 0.74
Validation Specificity: 0.08


##95% CI

In [ ]:
# Function to compute metrics
def compute_metrics(y_true, y_pred, y_pred_proba):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=1)
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    roc_auc = roc_auc_score(y_true, y_pred_proba)

    # Confusion matrix to compute specificity
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)

    return accuracy, precision, recall, f1, roc_auc, specificity

# Number of bootstrap iterations
n_iterations = 1000
n_size = len(X_val_ibd_rf)

# Initialize lists to store metric values for each bootstrap sample
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
roc_auc_scores = []
specificity_scores = []

# Bootstrap procedure
for i in range(n_iterations):
    # Resample the validation set with replacement
    X_resample, y_resample = resample(X_val_ibd_rf, y_encoded, n_samples=n_size, random_state=i)

    # Make predictions on the resampled data
    y_pred_resample = final_model_rf.predict(X_resample)
    y_pred_proba_resample = final_model_rf.predict_proba(X_resample)[:, 1]

    # Calculate metrics for this bootstrap sample
    accuracy, precision, recall, f1, roc_auc, specificity = compute_metrics(y_resample, y_pred_resample, y_pred_proba_resample)

    # Store the metrics
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)
    specificity_scores.append(specificity)

# Calculate 95% confidence intervals for each metric
def calculate_confidence_interval(scores):
    lower_bound = np.percentile(scores, 2.5)
    upper_bound = np.percentile(scores, 97.5)
    return lower_bound, upper_bound

# Calculate and print 95% confidence intervals
accuracy_ci = calculate_confidence_interval(accuracy_scores)
precision_ci = calculate_confidence_interval(precision_scores)
recall_ci = calculate_confidence_interval(recall_scores)
f1_ci = calculate_confidence_interval(f1_scores)
roc_auc_ci = calculate_confidence_interval(roc_auc_scores)
specificity_ci = calculate_confidence_interval(specificity_scores)

print(f'Validation ROC AUC: 95% CI: [{roc_auc_ci[0]:.2f}, {roc_auc_ci[1]:.2f}]')
print(f'Validation Accuracy:  95% CI: [{accuracy_ci[0]:.2f}, {accuracy_ci[1]:.2f}]')
print(f'Validation Precision: 95% CI: [{precision_ci[0]:.2f}, {precision_ci[1]:.2f}]')
print(f'Validation Recall: 95% CI: [{recall_ci[0]:.2f}, {recall_ci[1]:.2f}]')
print(f'Validation F1-Score:  95% CI: [{f1_ci[0]:.2f}, {f1_ci[1]:.2f}]')
print(f'Validation Specificity: 95% CI: [{specificity_ci[0]:.2f}, {specificity_ci[1]:.2f}]')


Validation ROC AUC: 95% CI: [0.50, 0.64]
Validation Accuracy:  95% CI: [0.68, 0.77]
Validation Precision: 95% CI: [0.49, 0.80]
Validation Recall: 95% CI: [0.68, 0.77]
Validation F1-Score:  95% CI: [0.68, 0.77]
Validation Specificity: 95% CI: [0.00, 0.03]


#LASSO

In [ ]:
X = gene.drop(['Group'], axis=1)
y = gene['Group']

# Encode categorical target labels into numerical labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Create Logistic Regression classifier with L1 regularization (Lasso)
log_reg = LogisticRegression(penalty='l1', solver='liblinear', random_state=42)

# Train the classifier
log_reg.fit(X_train, y_train)

# Predict on the test set
y_pred = log_reg.predict(X_test)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print the evaluation metrics
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

Accuracy: 0.7455
Precision: 0.8462
Recall: 0.8049
F1 Score: 0.8250


##Random Search

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Define the parameter grid for Randomized Search
param_dist = {
    'C': np.logspace(-4, 4, 20),
    'solver': ['liblinear'],
    'max_iter': [1000, 5000, 10000, 20000],
    'tol': [1e-4, 1e-3, 1e-2, 1e-1]
}

# Create Logistic Regression classifier with L1 regularization
log_reg = LogisticRegression(penalty='l1', random_state=42)

# Set up the Randomized Search with cross-validation
random_search = RandomizedSearchCV(
    log_reg, param_distributions=param_dist, n_iter=100,
    scoring='roc_auc', cv=5, verbose=1, random_state=42, n_jobs=-1
)

# Fit the Randomized Search model
random_search.fit(X_train, y_train)

# Get the best parameters
best_params = random_search.best_params_
print("Best Parameters:", best_params)

# Predict on the test set with the best model
y_pred = random_search.predict(X_test)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Print the evaluation metrics
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')


Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best Parameters: {'tol': 0.001, 'solver': 'liblinear', 'max_iter': 20000, 'C': 0.615848211066026}
Accuracy: 0.7636
Precision: 0.7821
Recall: 0.7636
F1 Score: 0.7706


##Bayesian Optimization

In [ ]:
# Random Search best parameters
random_search_params = {'tol': 0.001, 'solver': 'liblinear', 'max_iter': 20000, 'C':0.615848211066026}

def objective(trial):
    C = trial.suggest_float('C', 1e-4, 1e3, log=True)
    max_iter = trial.suggest_int('max_iter', 1000, 100000)
    tol = trial.suggest_float('tol', 1e-4, 1e-2, log=True)
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga'])
    clf = LogisticRegression(
        penalty='l1', C=C, max_iter=max_iter, tol=tol, solver=solver, random_state=42
    )
    score = cross_val_score(clf, X_train_balanced, y_train_balanced, cv=5, scoring='roc_auc').mean()
    return score

# Create the study
study = optuna.create_study(direction='maximize')

# Define the initial random search trial
def random_search_trial(trial):
    trial.suggest_float('C', 206.913808111479, 206.913808111479, log=True)
    trial.suggest_int('max_iter', 5000, 5000)
    trial.suggest_float('tol', 0.0001, 0.0001, log=True)
    trial.suggest_categorical('solver', ['liblinear'])

# Start the optimization
study.enqueue_trial(random_search_params)
study.optimize(objective, n_trials=50)

# Print the best parameters and best AUC-ROC score from Optuna
print("Best Parameters from Optuna:", study.best_params)
print("Best AUC-ROC Score from Optuna:", study.best_value)

# Retrieve the best parameters and train the model on the balanced data
best_params = study.best_params
clf = LogisticRegression(
    penalty='l1', **best_params, random_state=42
)
clf.fit(X_train_balanced, y_train_balanced)

# Predict on the test set
y_pred_prob = clf.predict_proba(X_test)[:, 1]
y_pred = clf.predict(X_test)

# Evaluate the model on the test set
roc_auc = roc_auc_score(y_test, y_pred_prob)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Print evaluation metrics
print(f"Test AUC-ROC: {roc_auc:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1 Score: {f1:.4f}")


[I 2025-01-08 12:32:56,592] A new study created in memory with name: no-name-43f52307-6433-4507-9ae4-235b71c7a790
[I 2025-01-08 12:32:56,694] Trial 0 finished with value: 0.9185733333333334 and parameters: {'C': 0.615848211066026, 'max_iter': 20000, 'tol': 0.001, 'solver': 'liblinear'}. Best is trial 0 with value: 0.9185733333333334.
[I 2025-01-08 12:32:56,796] Trial 1 finished with value: 0.91924 and parameters: {'C': 0.2591818985497918, 'max_iter': 29167, 'tol': 0.004319761784026923, 'solver': 'saga'}. Best is trial 1 with value: 0.91924.
[I 2025-01-08 12:32:57,334] Trial 2 finished with value: 0.9209733333333332 and parameters: {'C': 83.10784503953826, 'max_iter': 83268, 'tol': 0.00023864888482187419, 'solver': 'saga'}. Best is trial 2 with value: 0.9209733333333332.
[I 2025-01-08 12:32:57,486] Trial 3 finished with value: 0.9205066666666667 and parameters: {'C': 7.834541398759501, 'max_iter': 45993, 'tol': 0.0001545862564113279, 'solver': 'liblinear'}. Best is trial 2 with value: 0

Best Parameters from Optuna: {'C': 190.87285277863643, 'max_iter': 33719, 'tol': 0.009596604609187667, 'solver': 'saga'}
Best AUC-ROC Score from Optuna: 0.9246399999999999
Test AUC-ROC: 0.7474
Test Accuracy: 0.7091
Test Precision: 0.7535
Test Recall: 0.7091
Test F1 Score: 0.7233


In [ ]:
selected_features_lasso = X_train.columns.tolist()

# Best parameters from Bayesian Optimization with Optuna
best_params_lasso = {
    'C': 190.87285277863643,
    'max_iter': 33719,
    'solver': 'saga',
    'tol':0.009596604609187667
}

# Train the Logistic Regression model again using only the selected features
final_model_lasso = LogisticRegression(penalty='l1', **best_params_lasso, random_state=42)

# Perform cross-validation with 5 folds
cv_scores = cross_val_score(final_model_lasso, X_train[selected_features_lasso], y_train, cv=5, scoring='roc_auc')
print("Cross-validation scores:")
print(cv_scores)
print(f"Mean CV accuracy: {np.mean(cv_scores):.4f}")

final_model_lasso.fit(X_train[selected_features_lasso], y_train)

# Make predictions on the test set (predicted probabilities)
y_pred_proba_lasso = final_model_lasso.predict_proba(X_test[selected_features_lasso])[:, 1]

# Convert probabilities to predicted class labels
y_pred_lasso = final_model_lasso.predict(X_test[selected_features_lasso])


# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred_lasso)
precision = precision_score(y_test, y_pred_lasso, average = 'weighted')
recall = recall_score(y_test, y_pred_lasso, average = 'weighted')
f1 = f1_score(y_test, y_pred_lasso, average = 'weighted')
roc_auc = roc_auc_score(y_test, y_pred_proba_lasso)

# Calculate confusion matrix and extract TN, FP, FN, TP
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_lasso).ravel()

# Calculate specificity
specificity = tn / (tn + fp)

print(f'Test ROC AUC: {roc_auc:.2f}')
print(f'Test Accuracy: {accuracy:.2f}')
print(f'Test Precision: {precision:.2f}')
print(f'Test Recall: {recall:.2f}')
print(f'Test F1-Score: {f1:.2f}')
print(f'Test Specificity: {specificity:.2f}')



Cross-validation scores:
[0.8        0.81632653 0.85714286 0.81632653 0.87755102]
Mean CV accuracy: 0.8335
Test ROC AUC: 0.75
Test Accuracy: 0.71
Test Precision: 0.75
Test Recall: 0.71
Test F1-Score: 0.72
Test Specificity: 0.64


## LASSO Validation

In [ ]:
# Validation set
X_val_ibd_lasso = gene_ibd.drop('Group', axis=1)
y_val_ibd_lasso = gene_ibd['Group']

# Encode categorical target labels into numerical labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_val_ibd_lasso)

# Identify the features that are missing in the validation set
missing_features = [feature for feature in selected_features_lasso if feature not in X_val_ibd_lasso.columns]

# Add the missing features to the validation set with zero values using pd.concat
missing_df_lasso = pd.DataFrame(0, index=X_val_ibd_lasso.index, columns=missing_features)
X_val_ibd_lasso = pd.concat([X_val_ibd_lasso, missing_df_lasso], axis=1)

# Ensure the columns are in the same order as the training features
X_val_ibd_lasso = X_val_ibd_lasso[selected_features_lasso]

#Make predictions on the validation set
y_pred_prob_lasso_ibd= final_model_lasso.predict_proba(X_val_ibd_lasso)[:, 1]
y_pred_lasso_ibd = final_model_lasso.predict(X_val_ibd_lasso)

# Calculate evaluation metrics
accuracy_val = accuracy_score(y_encoded, y_pred_lasso_ibd)
precision_val = precision_score(y_encoded, y_pred_lasso_ibd)
recall_val = recall_score(y_encoded, y_pred_lasso_ibd, average="weighted")
f1_val = f1_score(y_encoded, y_pred_lasso_ibd,  average="weighted")
roc_auc_val = roc_auc_score(y_encoded, y_pred_prob_lasso_ibd)

# Calculate specificity
tn_val, fp_val, fn_val, tp_val = confusion_matrix(y_encoded, y_pred_lasso_ibd).ravel()
specificity_val = tn_val / (tn_val + fp_val)

# Print evaluation metrics
print(f'Validation ROC AUC: {roc_auc_val:.2f}')
print(f'Validation Accuracy: {accuracy_val:.2f}')
print(f'Validation Precision: {precision_val:.2f}')
print(f'Validation Recall: {recall_val:.2f}')
print(f'Validation F1-Score: {f1_val:.2f}')
print(f'Validation Specificity: {specificity_val:.2f}')

Validation ROC AUC: 0.53
Validation Accuracy: 0.70
Validation Precision: 0.74
Validation Recall: 0.70
Validation F1-Score: 0.70
Validation Specificity: 0.12


##95% CI

In [ ]:
def bootstrap_ci(metric_func, y_true, y_pred, y_proba=None, n_bootstraps=1000, alpha=0.95):
    bootstrapped_scores = []
    n_size = len(y_true)

    for i in range(n_bootstraps):
        indices = resample(range(n_size), replace=True, n_samples=n_size, random_state=i)
        y_true_resampled = y_true[indices]
        y_pred_resampled = y_pred[indices]

        if y_proba is not None:
            y_proba_resampled = y_proba[indices]
            score = metric_func(y_true_resampled, y_proba_resampled)
        else:
            score = metric_func(y_true_resampled, y_pred_resampled)

        bootstrapped_scores.append(score)

    # Compute the confidence interval
    sorted_scores = np.sort(bootstrapped_scores)
    lower_bound = np.percentile(sorted_scores, (1 - alpha) / 2 * 100)
    upper_bound = np.percentile(sorted_scores, (1 + alpha) / 2 * 100)

    return lower_bound, upper_bound

# Apply bootstrap CI for each metric
accuracy_ci = bootstrap_ci(accuracy_score, y_test_lasso, y_pred_lasso)
precision_ci = bootstrap_ci(precision_score, y_test_lasso, y_pred_lasso)
recall_ci = bootstrap_ci(recall_score, y_test_lasso, y_pred_lasso)
f1_ci = bootstrap_ci(f1_score, y_test_lasso, y_pred_lasso)
roc_auc_ci = bootstrap_ci(roc_auc_score, y_test_lasso, y_pred_lasso, y_proba=y_pred_proba_lasso)

# Specificity calculation
def specificity_metric(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)

specificity_ci = bootstrap_ci(specificity_metric, y_test_lasso, y_pred_lasso)

# Print evaluation metrics with their 95% Confidence Intervals
print(f'Test ROC AUC: {roc_auc:.2f} (95% CI: {roc_auc_ci[0]:.2f}, {roc_auc_ci[1]:.2f})')
print(f'Test Accuracy: {accuracy:.2f} (95% CI: {accuracy_ci[0]:.2f}, {accuracy_ci[1]:.2f})')
print(f'Test Precision: {precision:.2f} (95% CI: {precision_ci[0]:.2f}, {precision_ci[1]:.2f})')
print(f'Test Recall: {recall:.2f} (95% CI: {recall_ci[0]:.2f}, {recall_ci[1]:.2f})')
print(f'Test F1-Score: {f1:.2f} (95% CI: {f1_ci[0]:.2f}, {f1_ci[1]:.2f})')
print(f'Test Specificity: {specificity:.2f} (95% CI: {specificity_ci[0]:.2f}, {specificity_ci[1]:.2f})')


Validation ROC AUC: (95% CI: 0.46, 0.59)
Validation Specificity:(95% CI: 0.07, 0.19)
Validation Accuracy: (95% CI: 0.69, 0.78)
Validation Precision:  (95% CI: 0.79, 0.83)
Validation Recall: (95% CI: 0.69, 0.78)
Validation F1-Score:  (95% CI: 0.57, 0.69)
